## Setup

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip uninstall -f gym
!pip install -f gym==0.21.0 
!pip3 install torch torchvision torchaudio 
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 198 kB 32.5 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 51.0 MB/s 
  Created wheel for nes-py: filename=nes_py-8.2.1-cp37-cp37m-linux_x86_64.whl size=435502 sha256=a170cd52e9c3e20e377cd21c6553cb44625ba74435887c2bc2cf6b3940942c68
  Stored in directory: /root/.cache/pip/wheels/17/96/0e/22a8c7dbdf412d8e988286f223b223baf0f4ad90c9e699c56d
Successfully built nes-py
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gym-0.25.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gym/*
Proceed (y/n)? y
  Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 32.8 MB/s 
  Created wheel for gym: f

In [ ]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT 

from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

import os
# import algorithms
from stable_baselines3 import PPO, A2C, DQN
# import bse callback for saving models
from stable_baselines3.common.callbacks import BaseCallback 

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
done = True
# Loop through each frame in the game
for step in range(10000):
    # Start the game to begin with
    if done:
        # Start the game
        state = env.reset()
        # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    print(state.shape, reward, done, info)
    break
    # Show the game on the screen
    # env.render()
#close the game
env.close()

(240, 256, 3) 0.0 False {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}


# Models

## PPO

In [ ]:
#1 Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

#2 Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#3 Greyscale environment
env = GrayScaleObservation(env, keep_dim=True)

#4 Wrap inside dummy environment
env = DummyVecEnv([lambda: env])

#5 Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Create callback that allows you to save models
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [ ]:
# Setup directories
CHECKPOINT_DIR = '/content/'
LOG_DIR = '/content/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
# Setup the AI model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

In [ ]:
# Reloading a trained model
# model = PPO.load("./content/model", env=env)

In [ ]:
# Train the AI model
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
# Save Model
# model.save("/content/PPO/model")

# Load Model
# model = PPO.load('/content/PPO/model')

In [ ]:
# Start the game
state = env.reset()
done = False
score = 0

# Loop through game
while not done:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        score += reward
        # env.render()
print(score)

## A2C

In [ ]:
#1 Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

#2 Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#3 Greyscale environment
env = GrayScaleObservation(env, keep_dim=True)

#4 Wrap inside dummy environment
env = DummyVecEnv([lambda: env])

#5 Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Create callback that allows you to save models
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [ ]:
# Setup directories
CHECKPOINT_DIR = '/content/'
LOG_DIR = '/content/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
# Setup the AI model
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

In [ ]:
# Reloading a trained model
# model = A2C.load("./content/model", env=env)

In [ ]:
# Train the AI model
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
# Save Model
# model.save("/content/A2C/model")

# Load Model
# model = A2C.load('/content/A2C/model')

In [ ]:
# Start the game
state = env.reset()
done = False
score = 0

# Loop through game
while not done:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        score += reward
        # env.render()
print(score)

## DQN

In [ ]:
#1 Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

#2 Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#3 Greyscale environment
env = GrayScaleObservation(env, keep_dim=True)

#4 Wrap inside dummy environment
env = DummyVecEnv([lambda: env])

#5 Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Create callback that allows you to save models
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [ ]:
# Setup directories
CHECKPOINT_DIR = '/content/'
LOG_DIR = '/content/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
# Setup the AI model
model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, buffer_size=10000)

In [ ]:
# Reloading a trained model
# model = DQN.load("./content/model", env=env)

In [ ]:
# Train the AI model
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
# Save Model
# model.save("/content/DQN/model")

# Load Model
# model = DQN.load('/content/DQN/model')

In [ ]:
# Start the game
state = env.reset()
done = False
score = 0

# Loop through game
while not done:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        score += reward
        # env.render()
print(score)